### Pacman - Naive Bayes Shield + Firepits + Route Tracker

In [1]:
import random
import tkinter as tk
from tkinter import Canvas
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import resample
from collections import deque
from datetime import datetime
import copy
import time

###### Pacman Maps

In [2]:
# Simple Pacman Map
# 1 = wall, 2 = food, 0 = empty space
INITIAL_MAZE = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1],
    [1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1],
    [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1],
    [1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1],
    [1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]

# Firepit Maze
FIREPIT_MAZE = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 4, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1],
    [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1],
    [1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1],
    [1, 2, 0, 5, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 5, 5, 1, 1, 1, 0, 1, 1, 1],
    [1, 0, 0, 0, 0, 5, 5, 0, 1, 1, 1, 0, 1, 1, 1],
    [1, 0, 5, 0, 0, 5, 0, 0, 0, 0, 0, 0, 1, 1, 1],
    [1, 0, 5, 0, 0, 5, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 5, 0, 5, 5, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 5, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]


#### Utility Functions (Statistics, Time formating, Data importing/exporting)

In [2]:
def sum_rewards(reward_per_episode, episodes_completed):
    """Returns the reward for last 100 episodes for each episode"""
    sum_of_rewards = np.zeros(episodes_completed)
    for i in range(episodes_completed):
        # Rewards for last 100 episodes
        sum_of_rewards[i] = np.sum(reward_per_episode[max(0, i - 100):(i+1)])

    return sum_of_rewards

def cumulative_reward(rewards_per_episode):
    """Create a list of the cumulative reward the agent has gained per episode."""
    cumulative_reward = [rewards_per_episode[0]]
    for i in rewards_per_episode[1:]:
        cumulative_reward.append(i + cumulative_reward[-1])
        
    return cumulative_reward

def one_in_x(a_list, x):
    """Returns every one in x of a given list for purpose of smoothing reward graphs."""
    return a_list[::x]

def format_time(seconds):
    """Formats time in seconds into hours/minutes/seconds and returns a string of the resulting time."""
    minutes = int(seconds // 60)
    hours = int(minutes // 60)
    minutes = minutes % 60
    seconds = seconds % 60
    if hours > 0:
        return f"{hours} hours, {minutes} minute{'s' if minutes != 1 else ''}, {seconds:.2f} seconds"
    if minutes > 0:
        return f"{minutes} minute{'s' if minutes != 1 else ''}, {seconds:.2f} seconds"
    return f"{seconds:.2f} seconds"

def export_results(rewards, violations, save_name=None):
    """Function to export results to an Excel file."""
    # Create Pandas dataframe to hold results
    to_save = pd.DataFrame({"Rewards": rewards, "Violations": violations})
   
    # Export the DataFrame to Excel.
    if save_name:
        file_name = save_name + '.xlsx'
    else:
        file_name = "Results_" + datetime.now().strftime('%d.%m.%Y_%H.%M') + '.xlsx'
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
    to_save.to_excel(writer, sheet_name='Sheet1', index=False)
    writer.close()
    print(f"Save sucessful: {file_name}. Episodes: {len(to_save)}")

def import_results(file_name):
    """Function to load results and violations from Excel and return as lists."""
    df = pd.read_excel(file_name)
    print(f"{file_name} imported. Episodes: {len(df)}")    
    return df['Rewards'].tolist(), df["Violations"].tolist()

def dec4(n):
    """Convert a number to a number with four decimal places."""
    return float(f"{n:.4f}")

#### Agent Tracker

In [3]:
class AgentTracker():
    """Class to store the actions the agent has made at each coordinate.
    Can return the results in a coordinate grid for combination with original map."""
    def __init__(self):
        self.action_table = {}
    
    def add_action(self, coordinates, action):
        # Function to add an action and the cosesponding coordinates if they are not already present.
        if coordinates in self.action_table:
            self.action_table[coordinates][action] += 1 # Increment action count
        else:
            self.action_table[coordinates] = [0, 0, 0 , 0] # Initiate list of four zero counts for actions.
            self.action_table[coordinates][action] += 1 # Increment action count
    
    def action_grid(self):
        grid = [[0 for j in range(15)] for i in range(15)] # Initiate empty grid
        # 10, 11, 12, 13 represent the actions "North" "South", "East" and "West"
        for i in self.action_table:
            # For each coordinate, get the index of largest value add 10 and that becomes the grid value
            grid[i[0]][i[1]] = self.action_table[i].index(max(self.action_table[i])) + 10
   
        return grid


def visualize_maze(maze=INITIAL_MAZE, cell_size=30):
    """Visualize a Pac-Man maze using Tkinter. Written by Grok3"""
    # Calculate window size based on maze dimensions
    rows, cols = len(maze), len(maze[0])
    window_size = rows * cell_size

    # Set up Tkinter window
    root = tk.Tk()
    root.title("Pac-Man Maze Visualization")
    canvas = Canvas(root, width=window_size, height=window_size, bg="black")
    canvas.pack()
    
    # Pacman starting position is [1][1]
    maze[1][1] = 4
    
    # Draw the maze
    for row in range(rows):
        for col in range(cols):
            x1, y1 = col * cell_size, row * cell_size
            x2, y2 = x1 + cell_size, y1 + cell_size
            xc, yc = (x1 + x2) / 2, (y1 + y2) / 2  # Center point
            if maze[row][col] == 2:  # Food
                canvas.create_oval(x1 + 12, y1 + 12, x2 - 12, y2 - 12, fill="white")
            elif maze[row][col] == 4:  # Pacman Spawn
                canvas.create_oval(x1 + 20, y1 + 20, x2 - 20, y2 - 20, fill="red")
            elif maze[row][col] == 6:  # Ghost Spawn
                canvas.create_oval(x1 + 20, y1 + 20, x2 - 20, y2 - 20, fill="green")
            elif maze[row][col] == 1:  # Wall
                # Draw double blue lines for walls
                if row == 0 or maze[row-1][col] != 1:
                    canvas.create_line(x1, y1, x2, y1, fill="blue", width=4)
                    canvas.create_line(x1, y1 + 2, x2, y1 + 2, fill="blue", width=4)
                if row == rows - 1 or maze[row+1][col] != 1:
                    canvas.create_line(x1, y2, x2, y2, fill="blue", width=4)
                    canvas.create_line(x1, y2 - 2, x2, y2 - 2, fill="blue", width=4)
                if col == 0 or maze[row][col-1] != 1:
                    canvas.create_line(x1, y1, x1, y2, fill="blue", width=4)
                    canvas.create_line(x1 + 2, y1, x1 + 2, y2, fill="blue", width=4)
                if col == cols - 1 or maze[row][col+1] != 1:
                    canvas.create_line(x2, y1, x2, y2, fill="blue", width=4)
                    canvas.create_line(x2 - 2, y1, x2 - 2, y2, fill="blue", width=4)
            elif maze[row][col] == 5:  # Draw Firepit
                # Red base flame (largest)
                canvas.create_polygon(
                    xc - 10, y2 - 5,  # Bottom left
                    xc + 10, y2 - 5,  # Bottom right
                    xc + 8, yc,       # Right mid
                    xc + 2, y1 + 5,   # Top right
                    xc - 2, y1 + 8,   # Top left
                    xc - 8, yc,       # Left mid
                    fill="red", outline="red"
                )
                # Orange middle flame
                canvas.create_polygon(
                    xc - 7, y2 - 3,   # Bottom left
                    xc + 7, y2 - 3,   # Bottom right
                    xc + 5, yc + 2,   # Right mid
                    xc + 1, y1 + 8,   # Top right
                    xc - 1, y1 + 10,  # Top left
                    xc - 5, yc + 2,   # Left mid
                    fill="orange", outline="orange"
                )
                # Yellow inner flame (smallest)
                canvas.create_polygon(
                    xc - 4, y2 - 1,   # Bottom left
                    xc + 4, y2 - 1,   # Bottom right
                    xc + 3, yc + 4,   # Right mid
                    xc, y1 + 10,      # Top
                    xc - 3, yc + 4,   # Left mid
                    fill="yellow", outline="yellow"
                )
            elif maze[row][col] == 10:  # Up Arrow
                canvas.create_polygon(
                    xc, y1 + 10,          # Top point
                    xc - 10, y1 + 20,     # Bottom left
                    xc + 10, y1 + 20,     # Bottom right
                    fill="cyan", outline="magenta"
                )
            elif maze[row][col] == 11:  # Down Arrow
                canvas.create_polygon(
                    xc, y2 - 10,          # Bottom point
                    xc - 10, y2 - 20,     # Top left
                    xc + 10, y2 - 20,     # Top right
                    fill="cyan", outline="magenta"
                )
            elif maze[row][col] == 13:  # Left Arrow
                canvas.create_polygon(
                    x1 + 10, yc,          # Left point
                    x1 + 20, yc - 10,     # Top right
                    x1 + 20, yc + 10,     # Bottom right
                    fill="cyan", outline="magenta"
                )
            elif maze[row][col] == 12:  # Right Arrow
                canvas.create_polygon(
                    x2 - 10, yc,          # Right point
                    x2 - 20, yc - 10,     # Top left
                    x2 - 20, yc + 10,     # Bottom left
                    fill="cyan", outline="magenta"
                )


    # Start the Tkinter event loop
    root.mainloop()

def maze_and_route(maze, actions):
    """Function to combine a maze with a route for visualisation purposes."""
    maze = copy.deepcopy(maze)
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] not in [2, 5]:
                maze[i][j] = maze[i][j] + actions[i][j]
                
    return maze

#### Neural Bayes Shielding

In [ ]:
class NaiveBayesShield():
    """Shield class"""
    def __init__(self, mult=1, threshold=None):
        self.classifier = GaussianNB() # Initiate classifier
        self.train_count = 0
        self.mult = mult
        self.threshold = threshold
        
    
    def balance_samples(self, df, mult=1):    
        """Function to take in a dataframe of samples and balance the data by undersampling the safe states.
           Baseline make it a 1:1 Ratio.
           The mult paramater changes this. 2 means 2:1 - Safe:Unsafe
           Mult of 0 returns the original data"""
        # What it does: Randomly reduce the number of Class A samples to match Class B (e.g., 1,000 each). [Notes to delete]
        # Pros: Simple, reduces training time, balances classes perfectly.
        # Cons: Throws away a lot of data (26,000 Class A samples), potentially losing valuable patterns.
        if mult == 0:
            # Original data returned.
            return df

        unsafe_exps = df[df['Safe'] == 0]
        safe_exps = df[df['Safe'] == 1]

        # Undersample the safe experiences with no duplicates to match the number of unsafe ones present.
        undersampled_safe_exps = resample(safe_exps,replace=False, n_samples=int(len(unsafe_exps)*mult), random_state=6)

        # Combine the unsafe with undersampled safe.
        return pd.concat([unsafe_exps, undersampled_safe_exps])

    def train(self, data):
        # Function to train the classifier with new data.
        # Add column names to the data.
        data.columns = ["episode_number", "Pac_x", "Pac_y", "G1_x", "G1_y", "G2_x", "G2_y", "F_1", "F_2", "Action", "Safe"]
        
        # Balance data with input multiplier.
        balanced_data = self.balance_samples(data, self.mult)
          
        X_train = balanced_data.drop(balanced_data.columns[[0, 10]], axis=1) # Drop columns 0 and 10 (episode and safe)
        Y_train = balanced_data["Safe"]
        
        # Train Classifier with balanced data.
        self.classifier.partial_fit(X_train, Y_train, classes=[0, 1])
        
        self.train_count += 1 # Increment training count.
    
    def predict(self, exp):
        # Fuction to make a prediction on a data point.
        if self.threshold:
            # Use probability threshold (1 for safe class, 0 for unsafe class).
            Y_probs = self.classifier.predict_proba([exp])
            return (Y_probs[:, 1] >= self.threshold).astype(int)[0]
        else:
            return self.classifier.predict([exp])[0]
    
    def train_count(self):
        return self.train_count

In [ ]:
class ShieldMemory():
    """Class to store experiences:
    Format is (current_state, action, terminated)"""
    def __init__(self):
        self.memory = []
    
    def add_experience(self, exp):
        # Adds an experience to replay buffer.
        # Input format for exp: (state, action, safe, episode_number)
        # Experience format is a list of all of these combined. [episode_number, state, action, safe]
        experience = [exp[3]]
        for n in exp[0]:
            # Round to four decimal places
            experience.append(n) # dec4() not needed
        # experience += exp[0] 
        experience.append(exp[1])
        experience.append(exp[2])
        
        self.memory.append(experience)
    
    def current_memories(self):
        # Returns how many memories are currently stored.
        return len(self.memory)
    
    def export_memories(self):
        """Function to export memories to an Excel file."""
        # Create Pandas datafram to hold results
        df = pd.DataFrame(self.memory)
        # df.round(5) # Round to 5 decimal places to save storage/compute?
        df.columns = ["Ep_#", "Pac_x", "Pac_y", "G1_x", "G1_y", "G2_x", "G2_y", "F_1", "F_2", "Actn", "Safe"]

        # Export the DataFrame to Excel.
        df.to_csv("Shield_Experience_" + datetime.now().strftime('%d.%m.%Y_%H.%M')+".csv")
        print("Save sucessful")
        
    def wipe_memory(self):
        # Wipe memory.
        self.memory = []

### Deep Q-Learning

In [ ]:
class ExperienceReplay():
    """Class to store experiences for training purposes.
       Experiences are store in the format (current_state, action, new_state, reward, terminated)"""
    def __init__(self, memory_max=500000, batch_size=32):
        self.memory = deque([], maxlen=memory_max) # Maximum experiences stored
        self.batch_size = batch_size # Batch size
    
    def add_experience(self, exp):
        # Adds an experience to replay buffer.
        # Format (current_state, action, new_state, reward, terminated)
        self.memory.append(exp)
    
    def sample(self):
        # Returns a batch of samples from memory.
        return random.sample(self.memory, self.batch_size)
    
    def current_memories(self):
        # Returns how many memories are currently stored.
        return len(self.memory)


class DQN(nn.Module):
    """Class for the network model."""
    def __init__(self, in_states, out_actions):
        super().__init__()

        # Define network layers
        self.fc1 = nn.Linear(in_states, 32) 
        self.fc2 = nn.Linear(32, 16)
        self.out = nn.Linear(16, out_actions)

    def forward(self, x):
        # Forward pass through the network.
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.out(x)
        return x

class CoordinateMemory():
    # Class to store coordinates explored for exploration bonus purposes.
    def __init__(self):
        # Ititialise a set with Pacmans starting coordinates.
        self.memory = set((1, 1))
    
    def add(self, pos):
        # Adds an explored coordinate to memory and returns if it is newly explored.
        pos = tuple(pos)
        new = True # Is the coordinate newly explored, or not.
        if pos in self.memory:
            new = False
        self.memory.add(pos)
        
        return new

In [ ]:
# Directions for movement, including Stop
directions = {
    "North": [-1, 0],
    "South": [1, 0],
    "East": [0, 1],
    "West": [0, -1],
    "Stop": [0, 0]
}

# Relative directions for right-hand and left-hand rules
RIGHT_TURN = {
    "North": "East",
    "East": "South",
    "South": "West",
    "West": "North"
}

LEFT_TURN = {
    "North": "West",
    "West": "South",
    "South": "East",
    "East": "North"
}

OPPOSITE = {
    "North": "South",
    "South": "North",
    "East": "West",
    "West": "East"
}

# Game Settings
CELL_SIZE = 30
WINDOW_SIZE = len(INITIAL_MAZE) * CELL_SIZE
WALL_THICKNESS = 4
WALL_GAP = 2

# Ghost movement probability
GHOST_MOVE_PROBABILITY = 0.9


class WallFollowingGhost:
    """Class for the wall following ghosts movement policy. Largely generated by Grok 3
       Can be set to follow the left or right wall.
       
       Input: Colour, starting position, starting direction and follow side."""
    def __init__(self, color, start_pos, start_direction, follow_side="right"):
        self.color = color
        self.current_pos = start_pos
        self.direction = start_direction
        self.follow_side = follow_side

    def move(self, maze):
        row, col = self.current_pos
        if self.follow_side == "right":
            # Follow side is left.
            turn_dir = RIGHT_TURN[self.direction]
            straight_dir = self.direction
            left_dir = LEFT_TURN[self.direction]
            back_dir = OPPOSITE[self.direction]
            directions_to_try = [turn_dir, straight_dir, left_dir, back_dir]
        else:
            # Follow side is left
            turn_dir = LEFT_TURN[self.direction]
            straight_dir = self.direction
            right_dir = RIGHT_TURN[self.direction]
            back_dir = OPPOSITE[self.direction]
            directions_to_try = [turn_dir, straight_dir, right_dir, back_dir]

        for direction in directions_to_try:
            delta_row, delta_col = directions[direction]
            new_row = row + delta_row
            new_col = col + delta_col
            if (0 <= new_row < len(maze) and 0 <= new_col < len(maze[0]) and
                    maze[new_row][new_col] != 1):
                self.current_pos = [new_row, new_col]
                self.direction = direction
                break


class PacmanGame:
    """Class to handle and draw the Pacman game enviroment. Largely generated by Grok 3."""
    def __init__(self, root, num_ghosts=1, visualize=True):
        self.visualize = visualize
        self.root = root if visualize else None
        if visualize:
            self.canvas = Canvas(self.root, width=WINDOW_SIZE, height=WINDOW_SIZE, bg="black")
            self.canvas.pack()
        self.num_ghosts = max(1, min(4, num_ghosts))
        self.ghost_colors = ["red", "pink", "orange", "cyan"][:self.num_ghosts]
        self.ghost_start_positions = [[3, 5], [3, 10], [11, 4], [11, 10]][:self.num_ghosts] # Also set in self.ghosts
        self.ghosts = [
            # Red ghost (Blinky)
            WallFollowingGhost("red", [3, 5], "South", follow_side="right"),
            # Pink ghost (Pinky)
            WallFollowingGhost("pink", [3, 10], "South", follow_side="left"),
            # Orange ghost (Clyde)
            WallFollowingGhost("orange", [11, 4], "North", follow_side="right"),
            # Cyan ghost (Inky)
            WallFollowingGhost("cyan", [11, 10], "North", follow_side="left")
        ][:self.num_ghosts]
        self.ghost_positions = [ghost.current_pos for ghost in self.ghosts]
        self.ghost_modes = ["wall_following"] * self.num_ghosts
        self.state_size = 2 + 2 * self.num_ghosts
        self.action_size = 4
        self.actions = ["North", "South", "East", "West"] # Available actions
        self.reset_episode()

    def reset_episode(self):
        self.maze = [row[:] for row in INITIAL_MAZE]
        self.pacman_pos = [1, 1]  
        for i, ghost in enumerate(self.ghosts):
            ghost.current_pos = self.ghost_start_positions[i]
            if ghost.color == "red":
                ghost.current_pos = [3, 4]
                ghost.direction = "South"
            elif ghost.color == "pink":
                ghost.current_pos = [3, 10]
                ghost.direction = "South"
            elif ghost.color == "orange":
                ghost.current_pos = [11, 4]
                ghost.direction = "North"
            elif ghost.color == "cyan":
                ghost.current_pos = [11, 10]
                ghost.direction = "North"
        self.ghost_positions = [ghost.current_pos for ghost in self.ghosts]
        self.food_left = sum(row.count(2) for row in self.maze)  # Count only initial 2s
        self.running = True
        self.pacman_direction = "East"
        self.ghost_modes = ["wall_following"] * self.num_ghosts
        if self.visualize:
            self.draw_maze()
            self.draw_pacman()
            self.draw_ghosts()

    def draw_maze(self):
        if not self.visualize:
            return
        self.canvas.delete("all")
        rows, cols = len(self.maze), len(self.maze[0])
        for row in range(rows):
            for col in range(cols):
                x1, y1 = col * CELL_SIZE, row * CELL_SIZE
                x2, y2 = x1 + CELL_SIZE, y1 + CELL_SIZE
                if self.maze[row][col] == 2:
                    self.canvas.create_oval(x1 + 12, y1 + 12, x2 - 12, y2 - 12, fill="white")
                if self.maze[row][col] == 1:
                    if row == 0 or self.maze[row-1][col] != 1:
                        self.canvas.create_line(x1, y1, x2, y1, fill="blue", width=WALL_THICKNESS)
                        self.canvas.create_line(x1, y1 + WALL_GAP, x2, y1 + WALL_GAP, fill="blue", width=WALL_THICKNESS)
                    if row == rows - 1 or self.maze[row+1][col] != 1:
                        self.canvas.create_line(x1, y2, x2, y2, fill="blue", width=WALL_THICKNESS)
                        self.canvas.create_line(x1, y2 - WALL_GAP, x2, y2 - WALL_GAP, fill="blue", width=WALL_THICKNESS)
                    if col == 0 or self.maze[row][col-1] != 1:
                        self.canvas.create_line(x1, y1, x1, y2, fill="blue", width=WALL_THICKNESS)
                        self.canvas.create_line(x1 + WALL_GAP, y1, x1 + WALL_GAP, y2, fill="blue", width=WALL_THICKNESS)
                    if col == cols - 1 or self.maze[row][col+1] != 1:
                        self.canvas.create_line(x2, y1, x2, y2, fill="blue", width=WALL_THICKNESS)
                        self.canvas.create_line(x2 - WALL_GAP, y1, x2 - WALL_GAP, y2, fill="blue", width=WALL_THICKNESS)
                if self.maze[row][col] == 5:  # Draw Firepit
                    xc, yc = (x1 + x2) / 2, (y1 + y2) / 2  # Center point
                    # Red base flame (largest)
                    self.canvas.create_polygon(
                        xc - 10, y2 - 5,  # Bottom left
                        xc + 10, y2 - 5,  # Bottom right
                        xc + 8, yc,       # Right mid
                        xc + 2, y1 + 5,   # Top right
                        xc - 2, y1 + 8,   # Top left
                        xc - 8, yc,       # Left mid
                        fill="red", outline="red"
                    )
                    # Orange middle flame
                    self.canvas.create_polygon(
                        xc - 7, y2 - 3,   # Bottom left
                        xc + 7, y2 - 3,   # Bottom right
                        xc + 5, yc + 2,   # Right mid
                        xc + 1, y1 + 8,   # Top right
                        xc - 1, y1 + 10,  # Top left
                        xc - 5, yc + 2,   # Left mid
                        fill="orange", outline="orange"
                    )
                    # Yellow inner flame (smallest)
                    self.canvas.create_polygon(
                        xc - 4, y2 - 1,   # Bottom left
                        xc + 4, y2 - 1,   # Bottom right
                        xc + 3, yc + 4,   # Right mid
                        xc, y1 + 10,      # Top
                        xc - 3, yc + 4,   # Left mid
                        fill="yellow", outline="yellow"
                    )


    def draw_pacman(self):
        """Function to draw the Pacman in the Pacman game. Generated by Grok 3"""
        if not self.visualize:
            return
        row, col = self.pacman_pos
        x, y = col * CELL_SIZE + CELL_SIZE // 2, row * CELL_SIZE + CELL_SIZE // 2
        radius = CELL_SIZE // 2 - 2
        if self.pacman_direction == "East":
            start, extent = 30, 300
        elif self.pacman_direction == "West":
            start, extent = 210, 300
        elif self.pacman_direction == "North":
            start, extent = 120, 300
        else:
            start, extent = 300, 300
        self.pacman = self.canvas.create_arc(
            x - radius, y - radius, x + radius, y + radius,
            start=start, extent=extent, fill="yellow", outline="yellow"
        )

    def draw_ghosts(self):
        """Function to draw the ghots in the Pacman game. Generated by Grok 3"""
        if not self.visualize:
            return
        self.ghosts_drawn = []
        for i, ghost in enumerate(self.ghosts):
            row, col = ghost.current_pos
            x, y = col * CELL_SIZE + CELL_SIZE // 2, row * CELL_SIZE + CELL_SIZE // 2
            radius = CELL_SIZE // 2 - 2
            points = []
            num_points = 10
            for j in range(num_points + 1):
                angle = np.pi * j / num_points
                px = x - radius * np.cos(angle)
                py = y - radius * np.sin(angle)
                points.append(px)
                points.append(py)
            points.extend([
                x + radius, y + radius - 5,
                x + radius * 0.75, y + radius,
                x + radius * 0.25, y + radius - 5,
                x - radius * 0.25, y + radius,
                x - radius * 0.75, y + radius - 5,
                x - radius, y + radius - 5,
                x - radius, y
            ])
            body = self.canvas.create_polygon(points, fill=ghost.color, outline=ghost.color, smooth=True)
            eye_radius = radius // 3
            pupil_radius = eye_radius // 2
            eye_y_offset = y - radius * 0.5
            self.canvas.create_oval(x - radius * 0.5 - eye_radius, eye_y_offset - eye_radius,
                                    x - radius * 0.5 + eye_radius, eye_y_offset + eye_radius,
                                    fill="white")
            self.canvas.create_oval(x + radius * 0.5 - eye_radius, eye_y_offset - eye_radius,
                                    x + radius * 0.5 + eye_radius, eye_y_offset + eye_radius,
                                    fill="white")
            self.canvas.create_oval(x - radius * 0.5 - pupil_radius, eye_y_offset - pupil_radius,
                                    x - radius * 0.5 + pupil_radius, eye_y_offset + pupil_radius,
                                    fill="black")
            self.canvas.create_oval(x + radius * 0.5 - pupil_radius, eye_y_offset - pupil_radius,
                                    x + radius * 0.5 + pupil_radius, eye_y_offset + pupil_radius,
                                    fill="black")
            self.ghosts_drawn.append(body)

    def get_legal_directions(self, pos):
        legal_directions = []
        for direction, (delta_row, delta_col) in directions.items():
            if direction == "Stop":
                continue
            new_row = pos[0] + delta_row
            new_col = pos[1] + delta_col
            if 0 <= new_row < len(self.maze) and 0 <= new_col < len(self.maze[0]) and self.maze[new_row][new_col] != 1:
                legal_directions.append(direction)
        return legal_directions

    def move_ghost(self, ghost_idx, ghost_pos):
        if random.random() > GHOST_MOVE_PROBABILITY:
            return
        if ghost_idx >= self.num_ghosts:
            return

        self.ghosts[ghost_idx].move(self.maze)
        self.ghost_positions[ghost_idx] = self.ghosts[ghost_idx].current_pos

    def move_pacman(self, action):
        """Function to control the agents movemnets on the grid when an action is given."""
        delta_row, delta_col = directions[action]
        new_row = self.pacman_pos[0] + delta_row
        new_col = self.pacman_pos[1] + delta_col
        reward = 0
        if action != "Stop":
            self.pacman_direction = action
        if 0 <= new_row < len(self.maze) and 0 <= new_col < len(self.maze[0]) and self.maze[new_row][new_col] != 1:
            self.pacman_pos[0] = new_row
            self.pacman_pos[1] = new_col
            if self.maze[new_row][new_col] == 2:
                self.maze[new_row][new_col] = 0
                self.food_left -= 1
                reward = 0.8 # Reward for collecting food
        return reward


    def get_state(self, tensor=True):
        """Function to return a Tensor of the state of the environment.
           This is in the format of pacmans coordinates, then each ghosts coordinates,
           then if two pieces of food are present.  Written by PBN"""
        observations = [] # The state to be returned
        
        # Firepit Food                Simple Map Food
        #print(INITIAL_MAZE[6][11])   self.maze[3][7]
        # print(INITIAL_MAZE[7][1])   self.maze[1][12]
        
        # Add Pacmans's x and y coordinates.
        observations.append(self.pacman_pos[1]/14) # Pacmans's X position
        observations.append(self.pacman_pos[0]/6) # Pacmans's Y position
        # Add each Ghost's x and y coordinates. Normalised (with /14 and /6).
        for i, ghost_pos in enumerate(self.ghost_positions):
            observations.append(ghost_pos[1]/14)
            observations.append(ghost_pos[0]/6)
        # Add if each food is still present.
        if self.maze[6][11] == 2:
            observations.append(1) # Piece of food 1
        else:
            observations.append(0)
        if self.maze[7][1] == 2:
            observations.append(1) # Piece of food 2
        else:
            observations.append(0)

        if tensor:
            return torch.FloatTensor(observations)
        else:
            return observations
    
    def step(self, action):
        """Function to perform one step in the environment with the givven action.
           The position of the agent (Pacman) and the hazards (Ghosts) is updated.
           The status of any eaten food is also updated."""
        if self.running:
            reward = self.move_pacman(action) # Reward if food is eaten.
            for idx, ghost_pos in enumerate(self.ghost_positions):
                self.move_ghost(idx, ghost_pos)

            new_state = self.get_state()
            terminated = False
            if any(self.pacman_pos == ghost_pos for ghost_pos in self.ghost_positions):
                # Check if Pacman is in the same position as any ghosts.
                self.running = False
                terminated = True
            elif self.food_left == 0:
                # If food left is zero, Pacman has won the game.
                self.running = False
                terminated = True
            elif self.maze[self.pacman_pos[0]][self.pacman_pos[1]] == 5:  # Check for firepit
                # print("Hit Firepit at ", self.pacman_pos)
                self.running = False
                terminated = True

            if self.visualize:
                self.draw_maze()
                self.draw_pacman()
                self.draw_ghosts()

        return new_state, reward, terminated


def run_episodes(num_episodes, num_ghosts=1, visualize=True, move_limit=100, random_actions=False, DDQN=False, nb_shield=False, mult=1, threshold=None):
    # Training using the Pacman environment by Grok 3 with DQN/DDQN network.
    total_start_time = time.time()
    epsilon = 1 # E-greedy starting policy, starting at 100% randomised movement.
    decay = 0.0001 # Epsilon decay per episode [0.0001=10,000 episodes]
    root = tk.Tk() if visualize else None
    if visualize:
        root.title(f"Pac-Man with {num_ghosts} Ghost(s)")
    game = PacmanGame(root, num_ghosts, visualize)
    actions = ["North", "South", "East", "West"]
    
    # Statistics
    violations = 0 # Safety violations.
    timeouts = 0 # Amount of time the move limit is hit.
    reward_per_episode = []
    violations_per_episode = []
    
    # DQN
    loss_function = nn.MSELoss()
    memory = ExperienceReplay()
    discount_factor = 0.9 # The % of the reward that is propagated back to the the previous state.
    learning_rate = 0.001 # Learning rate for DQN.
    network_sync_rate = 100 # The rate at which the policy and target networks syncronise.
    input_len = 8
    num_actions = 4
    memory = ExperienceReplay() # Initialise Experience Replay.
    step_count = 0 # Steps between each target network update.

    # Create policy and target network
    policy_dqn = DQN(in_states=input_len, out_actions=num_actions)
    target_dqn = DQN(in_states=input_len, out_actions=num_actions)
    target_dqn.load_state_dict(policy_dqn.state_dict()) # Copy target networks weights from policy network.
    optimizer = torch.optim.Adam(policy_dqn.parameters(), lr=learning_rate) # Set Adam optimizer.
    
    # Shielding Functionality
    shield_memory = ShieldMemory() # Initialise Shield memory storage.
    shield = NaiveBayesShield(mult, threshold) # Initialise Shield
    
    # Initialise Route Tracker
    tracker = AgentTracker()
    
    def optimize(mini_batch, policy_dqn, target_dqn):
        # Function to perform one training step.
        current_q_list = [] # List of current calculated q values.
        target_q_list = [] # List of target q values.

        for state, action, new_state, reward, terminated in mini_batch:
            if terminated:
                if reward < 1:
                    # No exploration reward on death.
                    reward = 0
                target = torch.FloatTensor([reward])
            else:
                # Calculate Q value using Double DQN Algorithm.
                # Get best action from the policy network.
                if DDQN:
                    best_action = policy_dqn(state).argmax()
                    with torch.no_grad():
                        target = torch.FloatTensor(reward + discount_factor * target_dqn(new_state)[best_action])
                else: 
                    # Calculate Q value for DQN.
                    with torch.no_grad():
                        target = torch.FloatTensor(reward + discount_factor * target_dqn(new_state).max())
        
            # Get the current set of Q values
            current_q = policy_dqn(state)
            current_q_list.append(current_q)

            # Get the target set of Q values
            target_q = target_dqn(state)
            target_q[action] = target
            target_q_list.append(target_q)
        
        # Compute loss for the whole minibatch
        loss = loss_function(torch.stack(current_q_list), torch.stack(target_q_list))
        
        # Optimize the model
        optimizer.zero_grad()  # Zero the gradients
        loss.backward()
        optimizer.step()
        
    
    for episode in range(1, num_episodes + 1):
        # Train for required number of episodes.
        steps = 0
        total_reward = 0

        # Exploration Bonus Coordinate Storage
        exp_memory = CoordinateMemory()
        
        if (episode) % 50000 == 0:
            this_time = time.time()
            print(f"Time {format_time(time.time() - total_start_time)} Starting Episode {episode}...")
            
        game.reset_episode() # Reset environment to initial state.  
        state = game.get_state() # Get initial game state.
        while game.running:
            random_action = False
            if random.uniform(0, 1) > epsilon and not random_actions:
                # Select best action
                with torch.no_grad():
                    action_ind = policy_dqn(state).argmax().item()
                    if nb_shield:
                        # Get list of Actions sorted in descending Q-value order.
                        q_vals = policy_dqn(state).tolist() # Get Q-values for current state.
                        ind_q_vals = list(enumerate(q_vals)) # Enumerate to get index of each value.
                        sorted_indices = [ind for ind, val in sorted(ind_q_vals, key=lambda x: x[1], reverse=True)]
                    action = actions[action_ind]
            else:
                # Random action
                action = random.sample(actions, 1)[0]
                random_action = True
            
            """Shield Functionality"""
            if nb_shield:       
                # Get state for shield in list format.
                shield_state = game.get_state(False) 
            
            if nb_shield and shield.train_count > 0 and not random_action:
                # Only activate shield if it has been trained at least once and is not a random action.
                if not random_action:
                    for action_ind in sorted_indices:
                        # If no action deemed safe, original action will be taken
                        pred_state = shield_state.copy()
                        pred_state.append(action_ind)

                        # Make prediction of the safety of the action.
                        safe = shield.predict(pred_state)
                        if safe:
                            # If action is deemed safe, break loop, that action will be taken.
                            action = actions[action_ind]
                            break


            # Take action
            new_state, reward, terminated = game.step(action)
            
            new_exploration = exp_memory.add(game.pacman_pos)
            if new_exploration:
                # Extra reward is new coordinates explored.
                reward += 0.1

            total_reward += reward
            
            steps += 1 # Increment step counter
            step_count += 1 # Increment target network step counter
            
            if steps > move_limit:
                # Truncate episode if move limit is hit.
                game.running = False
            
            # Save experience to memory
            # Convert action back to it's relevant index.
            action_ind = actions.index(action)
            #Format (current_state, action, new_state, reward, terminated)
            memory.add_experience((state, action_ind, new_state, reward, terminated))
            
            # Add to Coordiantes and action to agent tracker
            tracker.add_action(tuple(game.pacman_pos), action_ind)
            
            if nb_shield:
                # Save experience to shield memory
                if terminated and reward < 0.2:
                    # If terminated and a low reward, must have hit a ghost
                    safe = 0
                else:
                    # 
                    safe = 1
                shield_memory.add_experience((shield_state, action_ind, safe, episode))

            # New state becomes current state.
            state = new_state
            
            if episode > 1000:
                # Decrease epsilon after each episode to min of 0.05.
                epsilon = max(epsilon - decay, 0.05)
            
            if visualize and root:
                root.update()
                root.after(100)

        
        # Check the outcome of the episode.
        if any(game.pacman_pos == ghost_pos for ghost_pos in game.ghost_positions):
            outcome = "Lost (caught by ghost)"
        elif game.food_left == 0:
            outcome = "Won (ate all food)"
        else:
            outcome = f"Exceeded {move_limit} moves"
    
        # Track reward and safety violations.
        reward_per_episode.append(total_reward)
        if outcome == "Lost (caught by ghost)":
            violations += 1 # Track violations.
            violations_per_episode.append(1)
        else:
            violations_per_episode.append(0)
        
        if outcome == f"Exceeded {move_limit} moves":
            timeouts += 1 # Track move limit violations.
            
        if not random_actions:
            for _ in range(5):
                # Five optimisations per episode.
                # Check if enough experience has been collected.
                if memory.current_memories() > 32:
                    mini_batch = memory.sample() # Collect a sample of data.
                    optimize(mini_batch, policy_dqn, target_dqn) # Train networks with sample.

                    # Update target network weights to policy network weights every x steps.
                    if step_count > network_sync_rate:
                        target_dqn.load_state_dict(policy_dqn.state_dict())
                        step_count = 0 # Reset counter
                    
        if episode % 10000 == 0 and nb_shield:
            #print(f"Training shield at episode {episode}, current samples {shield_memory.current_memories()}, {episode - violations - timeouts} episodes won.")
            # Convert current shield memory to a dataframe
            training_data = pd.DataFrame(shield_memory.memory)
            
            # Train shield with the data and wipe the memory as data has been used.
            shield.train(training_data)
            shield_memory.wipe_memory()
            #print(f"Training complete - Current samples {shield_memory.current_memories()}")
            
         
    # print("Shield memories:", shield_memory.current_memories())
    # print("Exporting shield data")
    # shield_memory.export_memories() ------------------------------------------------------------ MEMORY EXPORT FOR SHIELD
    
    # Create action grid for output
    route = tracker.action_grid()  
    
    total_end_time = time.time()
    total_duration = total_end_time - total_start_time
    #print(f"\nRuntime for {num_episodes} episodes: {format_time(total_duration)}")
    #print(f"{format_time(total_duration/num_episodes)} per episode")
    print(f"{num_episodes - violations - timeouts} episodes won.")
    print(f"Total safety violations: {violations}")
    print(f"Move limit hit: {timeouts}")
    print()
    if visualize and root:
        root.destroy()
    
    return reward_per_episode, violations, violations_per_episode, route

#### DQN Training

In [ ]:
# Settings
NUM_EPISODES = 200000 # Number of episodes to train for
TRAINING_RUNS = 1 # Number of training runs
NUM_GHOSTS = 2 # Number of ghosts present (2 simple, 3 in advanced)
VISUALIZE = False # Option to visualise training
MOVE_LIMIT = 800 # Maximum moves agent can make before episode truncation
RANDOM = False # Random actions toggle
DDQN = True # DDQN toggle (False reverts to DQN algorithm)
NAIVE_BAYES = True # Naive Bayes shield active
MULT = 0.5 # Data balancing multiplier
THRESHOLD = 0.52 # Safe class probability threshold
INITIAL_MAZE = FIREPIT_MAZE # Uncomment for Firepit maze usage

In [ ]:
# Import DDQN results for comparison
average_rewards, average_violations = import_results("Double_DQN_120k_3_runs.xlsx")

#### Naive Bayes Shield training

In [ ]:
print("Starting Naive Bayes Shield training...")
t1 = time.time()
all_rewards_NB = []
all_violations_NB = []
all_total_violations = 0

for i in range(TRAINING_RUNS):
    # Sum the runs and divide by number of runs to get the mean.
    rewards_ddqn, total_violations, violations_per_episode_NB, route = run_episodes(NUM_EPISODES, NUM_GHOSTS, VISUALIZE, MOVE_LIMIT, RANDOM, DDQN, NAIVE_BAYES, MULT, THRESHOLD)
    rew_100_ddqn = sum_rewards(rewards_ddqn, NUM_EPISODES)
    all_rewards_NB.append(rew_100_ddqn)
    all_violations_NB.append(violations_per_episode_NB)
    all_total_violations += total_violations 
    
average_rewards_NB = []
average_violations_NB = []
for i in range(NUM_EPISODES):
    # Sum the runs and divide by number of runs to get the mean.
    total = sum([j[i] for j in all_rewards_NB])
    total_v = sum([j[i] for j in all_violations_NB])
    average_rewards_NB.append(total/TRAINING_RUNS)
    average_violations_NB.append(total_v/TRAINING_RUNS)

t2 = time.time()
print("***** Training Ended *****")
print("Average violations per run: ", all_total_violations/TRAINING_RUNS)
print(f"Naive Bayes with DDQN training time: {format_time(t2 - t1)}")

# Export results
export_results(average_rewards_NB, average_violations_NB, "Naive_Bayes_FIREPIT_2_runs_3_runs")

In [ ]:
# Plot graph of results.
plt.rcParams['figure.figsize'] = [10, 4]
# plt.plot(average_rewards, label="DQN")
plt.plot(average_rewards_NB, label="Shielded DDQN") 
plt.ylim(0, 500)
plt.xlim(0, NUM_EPISODES)
plt.legend(loc="upper left")
plt.ylabel('Average Reward over last 100 Episodes')
plt.xlabel('Episode')
plt.show()

In [ ]:
# Zoomed in results
#plt.plot(average_rewards, label="DDQN")
plt.plot(average_rewards_NB, label="Shielded DDQN") 
plt.ylim(0, 500)
plt.xlim((NUM_EPISODES/5*4), NUM_EPISODES)
plt.legend(loc="lower right")
plt.ylabel('Average Reward over last 100 Episodes')
plt.xlabel('Episode')
plt.show()

In [ ]:
# Plot rewards and violations
figure, axis_1 = plt.subplots()
#axis_1.plot(average_rewards, 'b', label='DDQN')
axis_1.plot(average_rewards_NB, 'c--', label='Shielded')
axis_1.set_xlabel('Episode')
axis_1.set_ylabel('Average Reward over last 100 Episodes', color='b')
axis_2 = axis_1.twinx()
#axis_2.plot(cumulative_reward(average_violations), 'darkgreen', label='DQN')
axis_2.plot(cumulative_reward(average_violations_NB), 'g--', label='Shielded DDQN')
axis_2.set_ylabel('Cumulative Violations', color='g')
axis_1.legend(loc='upper left')
axis_2.legend(loc='lower right')
figure.tight_layout()
plt.show()

In [ ]:
# Plot rewards and violations with 1 in 1,000 points plotted.
x = 1000
figure, axis_1 = plt.subplots()
#axis_1.plot(one_in_x(average_rewards, x), 'b', label='DDQN')
axis_1.plot(one_in_x(average_rewards_NB, x), 'c--', label='Shielded DDQN')
axis_1.set_xlabel('Episode')
axis_1.set_ylabel('Average Reward over last 1000 Episodes', color='b')
axis_2 = axis_1.twinx()
#axis_2.plot(one_in_x(cumulative_reward(average_violations), x), 'darkgreen', label='DDQN')
axis_2.plot(one_in_x(cumulative_reward(average_violations_NB), x), 'g--', label='Shielded DDQN')
axis_2.set_ylabel('Cumulative Violations', color='g')
axis_1.legend(loc='upper left')
axis_2.legend(loc='lower right')
figure.tight_layout()
plt.show()

In [ ]:
# Add maze to route (most commonly taken actions at each coordinate)
MAZE = copy.deepcopy(FIREPIT_MAZE)
route_plus_maze = maze_and_route(FIREPIT_MAZE, route)

# Visualise the combined maze and most common action grid
visualize_maze(route_plus_maze, cell_size=30)